In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
from tqdm import tqdm

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
today = date.today()
today

datetime.date(2022, 7, 21)

In [4]:
from underwriting import data_quality_checks,client_surveillance_jupyter

In [5]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [6]:
ca_df = pd.read_csv('df_0720.csv',index_col = 0)
ca_df.columns = ['company_name','company_identifier','state','licenses']
ca_df['company_identifier'] = ca_df['company_identifier'].astype(str)

In [7]:
ca_df_clean = ca_df[~ca_df['company_identifier'].isin(['PL','WHT','GF','UR'])]
ca_df_clean.shape

(82, 4)

In [8]:
ca_df_clean['lic_list'] = [list(ca_df_clean['licenses'])[i].split(";") for i in range(len(ca_df_clean))]

In [9]:
ca_df_clean[0:3]

,company_name,company_identifier,state,licenses,lic_list
0,99 High Tide,99HT,CA,C10-0000279-LIC,[C10-0000279-LIC]
1,A Green Alternative Inc.,AGA,CA,C10-0000341-LIC,[C10-0000341-LIC]
2,Altum LLC,ALT,MI,PC-000310;AU-R-000380,"[PC-000310, AU-R-000380]"


# inventory analysis

In [10]:
today = date.today()
today

datetime.date(2022, 7, 21)

In [11]:
def read_inv_data(COMPANY_IDENTIFIER,license_numbers):
    
    company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=license_numbers,
    )
    company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)
    return company_inventory_packages_dataframe


In [ ]:
def get_age_bucket(age):
    if age < 90:
        return '0-90'
    elif 90 <= age < 270:
        return '90-9m'
    elif 270 <= age < 365:
        return '9m-12m'
    elif 365 <= age:
        return '>12m'
    else:
        return np.nan
        

In [12]:
def get_inv_list(company_list,map_df):
    inv_df_list = {}
    for c in tqdm(company_list):
        inv_list = {}
        print(c)
        l_list = map_df[map_df['company_identifier'] == c]['lic_list'].to_list()[0]
        for l in tqdm(l_list):
            print(l)
            #df_in = read_df_in([c],[l])
            df_inv = read_inv_data([c],[l])

            if df_inv.shape[0] == 0:
                inv_list[l] = numpy.nan
                continue
            else:
                #inv = client_surveillance_jupyter.calculate_inventory_valuation(df_in,df_inv,[l],today)
                #inv_list[l] = inv
#                 df_inv["age"] = [today - df_inv["packaged_date"][i] for i in range(len(df_inv))]
#                 df_inv["age_int"] = [df_inv["age"][i] / numpy.timedelta64(1, "D") for i in range(len(df_inv))]
#                 df_inv = df_inv.drop(['age'], axis=1)
#                 df_inv['age_bucket'] = [get_age_bucket(df_inv['age_int'][i]) for i in range(len(df_inv))]
                inv_list[l] = df_inv
        inv_df_list[c] =  inv_list
    return inv_df_list

In [13]:
b = get_inv_list(list(ca_df_clean['company_identifier']),ca_df_clean)




  0%|          | 0/82 [00:00<?, ?it/s]

99HT



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000279-LIC



  1%|          | 1/82 [00:01<02:33,  1.90s/it]

...


 33%|███▎      | 1/3 [00:01<00:03,  1.57s/it]

C10-0000802-LIC



 67%|██████▋   | 2/3 [00:03<00:01,  1.58s/it]

C12-0000159-LIC



100%|██████████| 82/82 [03:35<00:00,  2.63s/it]


In [14]:
b.keys()

dict_keys(['99HT', 'AGA', 'ALT', 'BS', 'BMC', 'BBF', 'BUD', 'BRC', 'BYN', 'CHO', 'CG', 'CPA', 'CPC', 'CSC', 'CSCC', 'CCC', 'CLC', 'CC', 'CED', 'DWF', 'DCO', 'DL', 'DGG', 'DG', 'EL', 'EMA', 'EMF', 'EMM', 'EMT', 'EPC', 'EH', 'FI', 'FW', 'GTC', 'GT', 'GLNR', 'GRG', 'GTR', 'GHC', 'GFEE', 'GFWF', 'HB', 'HS', 'HPCC', 'HC', 'IDC', 'ID', 'KAR', 'KC', 'LEG', 'LBC', 'MSS', 'ML', 'MW', 'MPW', 'MD', 'ND', 'NECC', 'PGM', 'PRP', 'PRL', 'QR', 'MC', 'RA', 'RGD', 'SGF', 'SFV', 'SV', 'SLCC', 'SO', 'ST', 'DW', 'FD', 'TGL', 'TJR', 'TTS', 'TT', 'T4L', 'TL', 'UHHC', 'VS', 'VOY'])

In [ ]:
df_inv = pd.DataFrame()
for k in b.keys():
    print(k)
    for l in b[k].keys():
        print(l)
        data_inv = b[k][l]
        if str(data_inv) == 'nan':
            continue
        else:
            data_inv['company'] = k
            data_inv['location'] = l
            print(data_inv.shape)
            df_inv = df_inv.append(data_inv)
df_inv['id'] = list(zip(df_inv.company, df_inv.location))
df_inv = df_inv.reset_index(drop = True)

## Age distribution by package count

In [36]:
df_clients = [
'DL',
'DW',
'EMA',
'EMM',
'EMT',
'EMF',
'ST',
'GRG',
'MPW',
'EL',
'VS',
'MD',
'DWF',
'TT',
'GHC',
'SV',
'99HT',
'NECC',
'LBC',
'BYN',
'CSC',
'CHO',
'BBF',
'CCC',
'PL',
'CPA'
]

In [ ]:
df_inv_df = df_inv[df_inv['company'].isin(df_clients)]

In [ ]:
age_package_count = df_inv_df[['id','age_bucket','package_id']].groupby(['id','age_bucket']).count().reset_index()  

In [ ]:
age_package_count['perc'] = age_package_count[['id','package_id']].groupby(['id']).apply(lambda x:100 * x / float(x.sum()))

In [ ]:
age_package_count.columns = ['id', 'age_bucket', 'count_pkg', 'perc']

In [ ]:
age_package_count.head()

In [ ]:
age_90 = age_package_count[age_package_count['age_bucket'] == '0-90']
age_909m = age_package_count[age_package_count['age_bucket'] == '90-9m']
age_9m12m = age_package_count[age_package_count['age_bucket'] == '9m-12m']
age_12m = age_package_count[age_package_count['age_bucket'] == '>12m']

In [ ]:
age_ = age_90.merge(age_909m,how = 'outer',on = 'id').\
merge(age_9m12m,how = 'outer',on = 'id').\
merge(age_12m,how = 'outer',on = 'id')

In [ ]:
age_.columns = ['id', 'age_bucket_90', 'count_pkg_90', 'perc_90', 
                     'age_bucket_909m','count_pkg_909m', 'perc_909m',
                    'age_bucket_9m12m','count_pkg_9m12m', 'perc_9m12m',
                     'age_bucket_12m','count_pkg_12m', 'perc_12m'
                    ]

In [ ]:
age_['new_fresh'] = age_['perc_90'] + age_['perc_909m']

In [ ]:
age_.head()

In [ ]:
age_.to_csv('df_inv_analysis_pkg.csv')

## Inventory valuation

In [15]:
def read_df_in(COMPANY_IDENTIFIER,license_numbers):
    company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
    )
    company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
    return company_incoming_transfer_packages_dataframe


In [40]:
def calculate_inventory_valuation_fresh(
    incoming_transfer_df, inventory_df, license_list, today_date,days_low,days_high
):
    # legal name
    legal_name = incoming_transfer_df[
        incoming_transfer_df["license_number"].isin(license_list)
    ]["recipient_facility_name"].values[0]
    # process df_in and df_sales
    incoming_transfer_df["per_unit_incoming"] = (
        incoming_transfer_df["shipper_wholesale_price"]
        / incoming_transfer_df["shipped_quantity"]
    )
    incoming_transfer_df_price = incoming_transfer_df[
        incoming_transfer_df["shipper_wholesale_price"].notnull()
    ]
    # by package id
    average_incoming_package_id = incoming_transfer_df_price.groupby(["package_id"])[
        "per_unit_incoming"
    ].mean()
    df_avg_incoming_price = pd.Series(average_incoming_package_id).to_frame()
    df_avg_incoming_price = df_avg_incoming_price.reset_index()
    # by product
    average_incoming_product = incoming_transfer_df_price.groupby(["product_name"])[
        "per_unit_incoming"
    ].mean()
    df_avg_product = pd.Series(average_incoming_product).to_frame()
    df_avg_product = df_avg_product.reset_index()
    df_avg_product.rename(
        columns={"per_unit_incoming": "per_unit_product"}, inplace=True
    )
    inventory_df = get_valid_inventory_df(inventory_df)

    # prepare fresh inventory
    inventory_df = inventory_df.reset_index(drop=True)
    inventory_df["age"] = [
        today_date - inventory_df["packaged_date"][i] for i in range(len(inventory_df))
    ]
    inventory_df["age_int"] = [
        inventory_df["age"][i] / numpy.timedelta64(1, "D")
        for i in range(len(inventory_df))
    ]
    inventory_df_fresh = inventory_df[(inventory_df["age_int"] > days_low)&(inventory_df["age_int"] <= days_high)]

    # calculate inventory
    df_inventory_incoming = pd.merge(
        inventory_df_fresh,
        df_avg_incoming_price,
        left_on=["package_id"],
        right_on=["package_id"],
        how="left",
    )
    # left_on=['tx_product_name','tx_unit_of_measure'], right_on=['product_name','shipped_unit_of_measure'], how='left'
    df_inventory_incoming.replace([numpy.inf], numpy.nan, inplace=True)
    df_inv_null = df_inventory_incoming[
        df_inventory_incoming["per_unit_incoming"].isnull()
    ]
    df_inv_product = pd.merge(
        df_inv_null,
        df_avg_product,
        left_on=["product_name"],
        right_on=["product_name"],
        how="left",
    )
    df_inv_product.replace([numpy.inf], numpy.nan, inplace=True)
    df_inv_product_price = df_inv_product[df_inv_product["per_unit_product"].notnull()]
    df_inv_product_price["total_price"] = (
        df_inv_product_price["quantity"] * df_inv_product_price["per_unit_product"]
    )

    inventory_product_value = df_inv_product_price["total_price"].sum()
    df_inventory_incoming["total_price"] = (
        df_inventory_incoming["quantity"] * df_inventory_incoming["per_unit_incoming"]
    )
    inventory_value = df_inventory_incoming["total_price"].sum()
    total_inv_value = inventory_product_value + inventory_value
    total_inv_value_after_tax = (inventory_product_value + inventory_value) * 1.2625
    inv_count_product = df_inv_product_price["per_unit_product"].count()
    inv_count_incoming = df_inventory_incoming["per_unit_incoming"].count()
    inv_count_total = df_inventory_incoming["quantity"].count()
    inv_total_incoming = inv_count_product + inv_count_incoming
    inventory_coverage = inv_total_incoming / inv_count_total
    # prepare data
    data = [
        [today_date],
        [round(total_inv_value, 2)],
        [round(total_inv_value_after_tax, 2)],
        [inv_total_incoming],
        [inv_count_total],
        [round(inventory_coverage, 2)],
        [license_list],
        [legal_name],
    ]
    df_inventory_license = pd.DataFrame(data).T
    df_inventory_license.columns = [
        "date",
        "value"+str(days_high),
        "value_after_tax",
        "total_incoming",
        "total",
        "coverage",
        "license",
        "legal_name",
    ]
    return df_inventory_license, inventory_df

In [41]:
def get_valid_inventory_df(inventory_data):
    #exclude trade samples
    inventory_data = inventory_data[~inventory_data['is_trade_sample']]
    #exclude quantity zero packages
    inventory_data = inventory_data[inventory_data['quantity'] > 0]
    return inventory_data

In [42]:
def get_inv_val_list(company_list,map_df):
    inv_df_list_90 = {}
    inv_df_list_9m = {}
    inv_df_list_12m = {}
    inv_df_list_12m_plus = {}
    for c in tqdm(company_list):
        inv_list_90 = {}
        inv_list_9m = {}
        inv_list_12m = {}
        inv_list_12m_plus = {}
        print(c)
        l_list = map_df[map_df['company_identifier'] == c]['lic_list'].to_list()[0]
        for l in tqdm(l_list):
            print(l)
            df_in = read_df_in([c],[l])
            df_inv = read_inv_data([c],[l])
            if (df_inv.shape[0] == 0) or df_in.shape[0] == 0:
                inv_list_90[l] = numpy.nan
                inv_list_9m[l] = numpy.nan
                inv_list_12m[l] = numpy.nan
                inv_list_12m_plus[l] = numpy.nan
                continue
            else:
                inv_90 = calculate_inventory_valuation_fresh(df_in,df_inv,[l],today,0,180)[0]
                inv_9m = calculate_inventory_valuation_fresh(df_in,df_inv,[l],today,180,270)[0]
                inv_12m = calculate_inventory_valuation_fresh(df_in,df_inv,[l],today,270,365)[0]
                inv_12m_plus = calculate_inventory_valuation_fresh(df_in,df_inv,[l],today,365,10000)[0]
                inv_list_90[l] = inv_90
                inv_list_9m[l] = inv_9m
                inv_list_12m[l] = inv_12m
                inv_list_12m_plus[l] = inv_12m_plus
        inv_df_list_90[c] =  inv_list_90
        inv_df_list_9m[c] =  inv_list_9m
        inv_df_list_12m[c] =  inv_list_12m
        inv_df_list_12m_plus[c] =  inv_list_12m_plus
    return inv_df_list_90,inv_df_list_9m,inv_df_list_12m,inv_df_list_12m_plus




In [43]:
t1,t2,t3,t4 = get_inv_val_list(list(ca_df_clean['company_identifier']),ca_df_clean)




  0%|          | 0/82 [00:00<?, ?it/s]

99HT



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000279-LIC



  1%|          | 1/82 [00:02<03:34,  2.65s/it]

...


 33%|███▎      | 1/3 [00:02<00:04,  2.11s/it]

C10-0000802-LIC



 67%|██████▋   | 2/3 [00:03<00:01,  1.83s/it]

C12-0000159-LIC



100%|██████████| 82/82 [05:09<00:00,  3.77s/it]


In [44]:
t4.keys()

dict_keys(['99HT', 'AGA', 'ALT', 'BS', 'BMC', 'BBF', 'BUD', 'BRC', 'BYN', 'CHO', 'CG', 'CPA', 'CPC', 'CSC', 'CSCC', 'CCC', 'CLC', 'CC', 'CED', 'DWF', 'DCO', 'DL', 'DGG', 'DG', 'EL', 'EMA', 'EMF', 'EMM', 'EMT', 'EPC', 'EH', 'FI', 'FW', 'GTC', 'GT', 'GLNR', 'GRG', 'GTR', 'GHC', 'GFEE', 'GFWF', 'HB', 'HS', 'HPCC', 'HC', 'IDC', 'ID', 'KAR', 'KC', 'LEG', 'LBC', 'MSS', 'ML', 'MW', 'MPW', 'MD', 'ND', 'NECC', 'PGM', 'PRP', 'PRL', 'QR', 'MC', 'RA', 'RGD', 'SGF', 'SFV', 'SV', 'SLCC', 'SO', 'ST', 'DW', 'FD', 'TGL', 'TJR', 'TTS', 'TT', 'T4L', 'TL', 'UHHC', 'VS', 'VOY'])

In [45]:
def decompress_inv_dict(inv_dict):
    df_decompress = pd.DataFrame()
    for k in inv_dict.keys():
        print(k)
        for l in inv_dict[k].keys():
            print(l)
            data_inv_val = inv_dict[k][l]
            if str(data_inv_val) == 'nan':
                continue
            else:
                data_inv_val['company'] = k
                data_inv_val['location'] = l
                print(data_inv_val.shape)
                df_decompress = df_decompress.append(data_inv_val)
    return df_decompress
    

In [46]:
df_decompress_90 = decompress_inv_dict(t1)
df_decompress_9m = decompress_inv_dict(t2)
df_decompress_12m = decompress_inv_dict(t3)
df_decompress_12m_plus = decompress_inv_dict(t4)

99HT
C10-0000279-LIC
(1, 10)
AGA
C10-0000341-LIC
(1, 10)
ALT
PC-000310
(1, 10)
AU-R-000380
(1, 10)
BS
AU-R-000156
(1, 10)
PC-000185
(1, 10)
BMC
402-00930
(1, 10)
BBF
MR281525
(1, 10)
BUD
C9-0000444-LIC
(1, 10)
C9-0000464-LIC
(1, 10)
C9-0000467-LIC
(1, 10)
C9-0000167-LIC
(1, 10)
C9-0000399-LIC
(1, 10)
BRC
C10-0000603-LIC
(1, 10)
BYN
C10-0000528-LIC
(1, 10)
CHO
AU-R-000575
(1, 10)
CG
C9-0000157-LIC
(1, 10)
CPA
C9-0000348-LIC
(1, 10)
C10-0000461-LIC
(1, 10)
CPC
C9-0000056-LIC
(1, 10)
CSC
C10-0000670-LIC
(1, 10)
CSCC
C10-0000918-LIC
(1, 10)
CCC
C12-0000087-LIC
(1, 10)
CLC
C10-0000986-LIC
C12-0000318-LIC
(1, 10)
CC
C10-0000758-LIC
(1, 10)
CED
C9-0000094-LIC
(1, 10)
DWF
402R-00296
(1, 10)
402R-00804
(1, 10)
DCO
050-10052885D4C
(1, 10)
050-10070593E9E
(1, 10)
DL
MR283369
(1, 10)
DGG
C9-0000016-LIC
(1, 10)
DG
AU-R-000287
(1, 10)
AU-R-000359
(1, 10)
AU-R-000422
(1, 10)
AU-R-000470
(1, 10)
AU-R-000572
(1, 10)
PC-000485
EL
C9-0000370-LIC
(1, 10)
EMA
C10-0000774-LIC
(1, 10)
EMF
C10-0000939-LIC
(1,

UHHC
C10-0000817-LIC
(1, 10)
C9-0000082-LIC
(1, 10)
VS
402-00840
(1, 10)
402R-00545
(1, 10)
VOY
C12-0000030-LIC
(1, 10)
C10-0000802-LIC
(1, 10)
C12-0000159-LIC
(1, 10)
99HT
C10-0000279-LIC
(1, 10)
AGA
C10-0000341-LIC
(1, 10)
ALT
PC-000310
(1, 10)
AU-R-000380
(1, 10)
BS
AU-R-000156
(1, 10)
PC-000185
(1, 10)
BMC
402-00930
(1, 10)
BBF
MR281525
(1, 10)
BUD
C9-0000444-LIC
(1, 10)
C9-0000464-LIC
(1, 10)
C9-0000467-LIC
(1, 10)
C9-0000167-LIC
(1, 10)
C9-0000399-LIC
(1, 10)
BRC
C10-0000603-LIC
(1, 10)
BYN
C10-0000528-LIC
(1, 10)
CHO
AU-R-000575
(1, 10)
CG
C9-0000157-LIC
(1, 10)
CPA
C9-0000348-LIC
(1, 10)
C10-0000461-LIC
(1, 10)
CPC
C9-0000056-LIC
(1, 10)
CSC
C10-0000670-LIC
(1, 10)
CSCC
C10-0000918-LIC
(1, 10)
CCC
C12-0000087-LIC
(1, 10)
CLC
C10-0000986-LIC
C12-0000318-LIC
(1, 10)
CC
C10-0000758-LIC
(1, 10)
CED
C9-0000094-LIC
(1, 10)
DWF
402R-00296
(1, 10)
402R-00804
(1, 10)
DCO
050-10052885D4C
(1, 10)
050-10070593E9E
(1, 10)
DL
MR283369
(1, 10)
DGG
C9-0000016-LIC
(1, 10)
DG
AU-R-000287
(1, 10)

In [47]:
age_val = df_decompress_90[['company','location','value180']].\
merge(df_decompress_9m[['company','location','value270']],on = ['company','location'],how = 'outer').\
merge(df_decompress_12m[['company','location','value365']],on = ['company','location'],how = 'outer').\
merge(df_decompress_12m_plus[['company','location','value10000']],on = ['company','location'],how = 'outer')

age_val['id'] = list(zip(age_val.company, age_val.location))
age_val = age_val.reset_index(drop = True)



In [48]:
age_val

,company,location,value180,value270,value365,value10000,id
0,99HT,C10-0000279-LIC,118713.32,1569.5,3472.58,2948.87,"(99HT, C10-0000279-LIC)"
1,AGA,C10-0000341-LIC,98760.39,9804.85,19236.72,30161.77,"(AGA, C10-0000341-LIC)"
2,ALT,PC-000310,20983.15,7590.09,1526.04,2547.5,"(ALT, PC-000310)"
3,ALT,AU-R-000380,253153.36,35809.33,720.29,2511.7,"(ALT, AU-R-000380)"
4,BS,AU-R-000156,29281.56,3495.4,714.65,1022.0,"(BS, AU-R-000156)"
...,...,...,...,...,...,...,...
152,VS,402-00840,32485.72,897.0,88.54,147.0,"(VS, 402-00840)"
153,VS,402R-00545,97570.24,3307.8,12.0,0.0,"(VS, 402R-00545)"
154,VOY,C12-0000030-LIC,8156.18,307.84,35.11,0.2,"(VOY, C12-0000030-LIC)"
155,VOY,C10-0000802-LIC,55488.26,4820.08,335.48,0.0,"(VOY, C10-0000802-LIC)"


In [49]:
age_val_df = age_val[age_val['company'].isin(df_clients)]

In [50]:
age_val_df

,company,location,value180,value270,value365,value10000,id
0,99HT,C10-0000279-LIC,118713.32,1569.5,3472.58,2948.87,"(99HT, C10-0000279-LIC)"
7,BBF,MR281525,77968.47,5402.37,4324.67,60.0,"(BBF, MR281525)"
14,BYN,C10-0000528-LIC,61291.62,3854.71,1133.25,785.0,"(BYN, C10-0000528-LIC)"
15,CHO,AU-R-000575,64252.4,8958.65,703.54,90.7,"(CHO, AU-R-000575)"
17,CPA,C9-0000348-LIC,914633.47,569610.54,587305.54,1293364.73,"(CPA, C9-0000348-LIC)"
18,CPA,C10-0000461-LIC,266582.46,365.59,4101.59,1839.77,"(CPA, C10-0000461-LIC)"
20,CSC,C10-0000670-LIC,181593.31,82731.81,42750.99,468531.04,"(CSC, C10-0000670-LIC)"
22,CCC,C12-0000087-LIC,931495.68,489527.91,478774.75,2753202.46,"(CCC, C12-0000087-LIC)"
26,DWF,402R-00296,216637.23,3195.0,8721.0,2069.0,"(DWF, 402R-00296)"
27,DWF,402R-00804,30665.37,2945.63,4766.2,466.85,"(DWF, 402R-00804)"


In [52]:
age_val_df.to_csv('df_inv_analysis_val_0720.csv')

## Consider other bucket cutoffs?

In [ ]:
age_distr = df_inv_df[['company','location','age_int']].groupby(['company','location']).describe().reset_index()

In [ ]:
#age_distr.to_csv('df_inv_age_distr.csv')

## age by product category

In [ ]:
df_inv_df['company'].unique()

In [ ]:
df_inv_df_with_state = pd.merge(df_inv_df,ca_df_clean[['company_identifier','state']],left_on = 'company',right_on = 'company_identifier',how = 'left')



In [ ]:
age_distr_by_product = df_inv_df_with_state[['product_category_name','age_int']].groupby(['product_category_name']).describe()

In [ ]:
age_distr_by_product.to_csv('age_distr_by_product.csv')

In [ ]:
age_distr_by_product